In [24]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd

price_with_typhoon = pd.read_csv("../data/101_108_price_with_typhoon.csv")

## Target market : 台北一
## Target crops :
**['鳳梨-金鑽鳳梨', '香蕉', '青蔥-日蔥','蓮霧-紅蓮霧', '芽菜類-黃豆牙', '南瓜-木瓜形', '大蒜-蒜瓣', '小白菜-土白菜', '木瓜-網室紅肉', '洋香瓜-網狀紅肉']**

In [25]:
target_crops = ['鳳梨-金鑽鳳梨', '香蕉', '青蔥-日蔥','蓮霧-紅蓮霧', '芽菜類-黃豆牙', '南瓜-木瓜形', '大蒜-蒜瓣', \
                '小白菜-土白菜', '木瓜-網室紅肉', '洋香瓜-網狀紅肉']
crops_info_arr = []

for crops in target_crops:
    crops_info_arr.append(
        price_with_typhoon.loc[
            (price_with_typhoon['市場名稱'] == '台北一') & (price_with_typhoon['作物名稱'] == crops)
        ]
    )

## Concat target crops info with weather info

In [26]:
import glob

path = "../data/weather_data/*.csv"
weathers = glob.glob(path)

weather_df_arr = [pd.read_csv(weather, header=0) for weather in weathers]

weather_info = pd.concat(weather_df_arr, axis=0, ignore_index=True).sort_values(
    by=["Year", "Month", "Day"], ignore_index=True
)

weather_info["DateTime"] = pd.to_datetime(
    (weather_info.Year * 10000 + weather_info.Month * 100 + weather_info.Day).apply(
        str
    ),
    format="%Y%m%d",
).astype(str)


crops_info_arr = [
    pd.merge(crops_info, weather_info, how="inner", on=["DateTime", "Year", "Month", "Day"])
    for crops_info in crops_info_arr
]

### LabelEncoder
- **column : 強度**

### Handle missing value on weather column 
- **['X', 'T', '...', '/']**

### Drop meaningless column

In [27]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

def process(crops_info):
    crops_name = crops_info["作物名稱"][0]
    crops_info["強度"] = crops_info["強度"].astype(str)
    crops_info["強度"] = labelencoder.fit_transform(crops_info["強度"])
    crops_info = crops_info.fillna('0')

    crops_info = crops_info.drop(
        columns=[
            "入台時間",
            "離台時間",
            "作物代號",
            "作物名稱",
            "市場代號",
            "市場名稱",
        ]
    )
    for col in list(crops_info.columns):
        try:
            crops_info[col] = crops_info[col].astype(float)
        except:
            pass
            error = crops_info.loc[crops_info[col].isin(['X', 'T', '...', '/'])]
            for index in list(error.index):
                crops_info[col][index] = '0'
    return crops_info, crops_name

for crops_info in crops_info_arr:
    df, name = process(crops_info)
    df.to_csv('../data/{}.csv'.format(name), index = False, encoding='utf_8_sig')

<ipython-input-27-3e05d1a61580>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crops_info[col][index] = '0'
